# Miner rule


In [ ]:
from pylife import *

In [ ]:
from pylife.stress.rainflow import *
import pylife.strength.meanstress
from pylife.stress.timesignal import TimeSignalGenerator
from pylife.strength import miner
from pylife.strength.miner import MinerElementar, MinerHaibach

import numpy as np
import pandas as pd

In [ ]:
tsgen = TimeSignalGenerator(10, {'number': 50,
                                 'amplitude_median': 1.0, 'amplitude_std_dev': 0.5,
                                 'frequency_median': 4, 'frequency_std_dev': 3,
                                 'offset_median': 0, 'offset_std_dev': 0.4},
                            None, None)

y = tsgen.query(2000000)

rfc = RainflowCounterThreePoint().process(y)
rfm = rfc.get_rainflow_matrix_frame(128)

transformed = rfm.meanstress_hist.FKM_goodman(pd.Series({'M': 0.3, 'M2': 0.1}), R_goal=-1)

print(transformed)

In [ ]:
sn_curve_parameters = {
    "N_D": 1E6,
    "k": 6,
    "sigma_ak": 200
}

In [ ]:
miner_elementar = MinerElementar(**sn_curve_parameters)

In [ ]:
# sn-curve calculations can be performed using the sn_curve instance that is stored as attribute
miner_elementar.sn_curve

In [ ]:
miner_elementar.sn_curve.calc_N(200) == sn_curve_parameters["N_D"]

In [ ]:
# extrapolation beyond the application limits is possible but prohibited by default
try:
    miner_elementar.sn_curve.calc_S(10E6) == sn_curve_parameters["sigma_ak"]
except ValueError as e:
    display(e)
    miner_elementar.sn_curve.calc_S(10E6, ignore_limits=True) == sn_curve_parameters["sigma_ak"]

In [ ]:
miner_elementar.sn_curve.calc_S(1E6, ignore_limits=True) == sn_curve_parameters["sigma_ak"]

In [ ]:
miner_elementar.setup(transformed)

In [ ]:
miner_elementar.V_FKM

In [ ]:
miner_elementar.V_haibach

In [ ]:
miner_elementar.V_haibach**(1 / miner_elementar.k)

In [ ]:
miner_elementar.N_predict(load_level=200)

In [ ]:
miner_elementar.N_predict(load_level=180)

In [ ]:
miner_haibach = MinerHaibach(**sn_curve_parameters)

In [ ]:
miner_haibach.setup(transformed)

In [ ]:
try:
    miner_haibach.V_FKM
except AttributeError:
    print("The accumulated damage by Miner modified (Haibach) can not be \n"
         "expressed with the solidity. This parameter is estimated and stored \n"
         "only for Miner-elementary")

In [ ]:
miner_haibach.N_predict(load_level=200)

In [ ]:
miner_haibach.N_predict(load_level=199.999)

In [ ]:
miner_haibach.N_predict(load_level=180)